In [54]:
import json
import glob
import os
import tarfile
import re

In [28]:
PREFIX = "/home/jpivarski/storage/data/physicists/forks-of-cmssw-2022-03-04"

In [29]:
userrepos = []
for filename in glob.glob(PREFIX + "/user-repos/*.json"):
    userrepos.extend(json.load(open(filename)))

In [30]:
len(set([repo["full_name"] for repo in userrepos if not repo["fork"]]))

16737

In [31]:
len([repo for repo in userrepos if not repo["fork"]])

16737

In [113]:
regex = {
    ("py", r"\buproot[34]*\b"),
    ("py", r"\bawkward[01]*\b"),
    ("cpp", r"\bTFile\b"),
    ("py", r"\bTFile\b"),
    ("py", r"\bimport\s+ROOT\b"),
    ("py", r"\bfrom\s+ROOT\s+import\b"),
    ("py", r"\bnumpy\b"),
    ("py", r"\bscipy\b"),
    ("py", r"\bmatplotlib\b"),
    ("py", r"\bpandas\b"),
    ("py", r"\bsklearn\b"),
    ("py", r"\btensorflow\b"),
    ("py", r"\bkeras\b"),
    ("py", r"\btorch\b"),
    ("py", r"\bjax\b"),
    ("py", r"\bxgboost\b"),
    ("py", r"\bcoffea\b"),
    ("py", r"\bRDataFrame\b"),
    ("cpp", r"\bRDataFrame\b"),
    ("py", r"\bTMVA\b"),
    ("cpp", r"\bTMVA\b"),
}

In [114]:
for repo in userrepos:
    if not repo["fork"]:
        owner, reponame = repo["full_name"].split("/")
        created_at = datetime.datetime.fromisoformat(repo["created_at"].rstrip("Z"))
        matches = {x: False for x in regex}
        tarname = f"{PREFIX}/actual-repos-2/{owner}-{reponame}.tgz"
        if os.path.exists(tarname):
            with tarfile.open(tarname) as tarball:
                # with tarball.extractfile(f"{owner}-{reponame}/git-log.json") as file:
                #     ages = {}
                #     for commit in git_log["log"]:
                #         timestamp = commit["commit_time"]
                #         for delta in commit["deltas"]:
                #             if delta["new"] not in ages or timestamp > ages[delta["new"]]:
                #                 ages[delta["new"]] = timestamp
                for name in tarball.getnames():
                    if name.endswith(".py"):
                        with tarball.extractfile(name) as file:
                            source = file.read().decode()
                            print(name, created_at)
            break

rateixei-bbggPlotter/python/CfiFile_cfi.py 2015-06-16 18:50:11
rateixei-bbggPlotter/python/ConfFile_cfg.py 2015-06-16 18:50:11
rateixei-bbggPlotter/test/miniAOD_GravFiles.py 2015-06-16 18:50:11
rateixei-bbggPlotter/test/miniAOD_RadFiles.py 2015-06-16 18:50:11
rateixei-bbggPlotter/test/TEMPLATE_MakePlots.py 2015-06-16 18:50:11
rateixei-bbggPlotter/test/microAOD_RadFiles.py 2015-06-16 18:50:11
rateixei-bbggPlotter/test/microAOD_GravFiles.py 2015-06-16 18:50:11
rateixei-bbggPlotter/test/MakePlots.py 2015-06-16 18:50:11
